In [34]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalshelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "CS340Password123"
host = "nv-desktop-services.apporto.com"
port = 32845
database = "AAC"
collection = "animals"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png'
with open(image_filename, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
# Add the company logo
    html.Div([
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image),
            style={'height': '250px', 'width': 'auto', 'margin': '10px'}
        )
    ], style={'textAlign': 'center'}),  # Center the logo
    
    html.Center(html.B(html.H1('Jordan Mitchell Dashboard'))),  # Dashboard title
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
            {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
            {'label': 'All', 'value': 'All'}
        ],
        value='All',  # Default selection
        labelStyle={'display': 'inline-block'}
    )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=True,
        row_selectable="single",  # Allow selection of a single row
        selected_rows=[],
        filter_action="native",  # Enable filtering
        sort_action="native",  # Enable sorting
        page_action="native",  # Enable pagination
        page_current=0,  # Start on the first page
        page_size=10,  # Show 10 rows per page
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    try:
            # Define query based on filter type
            if filter_type == 'Water Rescue':
                query = {
                    "animal_type": "Dog",
                    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
                    "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                    "sex_upon_outcome": "Intact Female"
                }
            elif filter_type == 'Mountain Rescue':
                query = {
                    "animal_type": "Dog",
                    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
                    "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                    "sex_upon_outcome": "Intact Male"
                }
            elif filter_type == 'Disaster Rescue':
                query = {
                    "animal_type": "Dog",
                    "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
                    "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                    "sex_upon_outcome": "Intact Male"
                }
            else:
                query = {}  # Return all records

            # Retrieve and clean up the data
            filtered_df = pd.DataFrame.from_records(db.read(query))
            filtered_df.drop(columns=['_id'], inplace=True, errors='ignore')  # drops '_id' if present

            # return filtered data to DataTable
            return filtered_df.to_dict('records')

    except Exception as e:
            print(f"Error: {e}")
            # Return empty DataFrame if error occurs
            return pd.DataFrame().to_dict('records')
        
## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', "value")])
def update_graphs(viewData, filter_type):
    if viewData is None or len(viewData) == 0:
        return [dcc.Graph(figure=px.pie(names=['No Data'], values=[1], title='No Data'))]

    # Convert table data to DataFrame
    df = pd.DataFrame.from_dict(viewData)

    if 'breed' not in df.columns or df['breed'].isnull().all():
        return [dcc.Graph(figure=px.pie(names=['No Data'], values=[1], title='No Breed Data'))]

    # Group excess breeds into "Other" for 'All' filter for better visualization (less/unreadable slices)
    if filter_type == "All":
        breed_counts = df['breed'].value_counts()
        threshold = 0.01 * breed_counts.sum()  # 1% threshold of total

        # Breeds with counts less than threshold fall within 'Other'
        grouped_breeds = breed_counts[breed_counts >= threshold].index
        df['breed_grouped'] = df['breed'].apply(lambda x: x if x in grouped_breeds else "Other (less than 1%)")

        pie_chart = px.pie(
            df,
            names='breed_grouped',
            title='Distribution of All Breeds'
        )
    else:
        pie_chart = px.pie(df, names='breed', title=f'Distribution of Breeds ({filter_type})')

    return [dcc.Graph(figure=pie_chart)]
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    #]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Ensure selected_column is valid
    if not selected_columns:
        return []  # No styling applied when no column is selected
    
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
#Code for geolocation chart
    # Convert table data to DataFrame
    df = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    # Verify if a row is selected
    if viewData is None or index is None or len(index) == 0:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[30.75, -97.48], children=[
                dl.Tooltip("Austin Animal Center"),
                dl.Popup([
                    html.H4("Default Location: Austin Animal Center"),
                    html.P("Default location of the animal shelter")
                ])
            ])
        ])
    
    # Extract data from the selected row
    row = index[0]
    lat = float(viewData[row]['location_lat'])
    lon = float(viewData[row]['location_long'])
    name = viewData[row]['name']
    breed = viewData[row]['breed']
    
    # Return map with location marker
    return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=15, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(name),
            dl.Popup([
                html.H4("Animal Name"),
                html.P(name),
                html.P(breed)
            ])
        ])
    ])
app.run_server(debug=True)

Dash app running on http://127.0.0.1:27172/
